In [ ]:
import utils
import io
import numpy as np
import math
import scipy
import mpmath
import matplotlib.pyplot as plt
import seaborn as sns
from functools import partial
import bayesflow as bf
import tensorflow as tf
import tensorflow_probability as tfp
# Set TensorFlow to use only the CPU
tf.config.set_visible_devices([], 'GPU')

cor, pal = utils.matplotlib_style()

# Set random seed
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

# Two-state promoter distribution

In this notebook, we will explore training a `BayesFlow` neural network to
perform parametric inference on the biophysical parameters of a two-state
promoter model. The two-state promoter model, also known as the "telegraph
model", is a simple model of gene expression that describes the dynamics of a
promotion region that can be in one of two states: active or inactive. The
steady mRNA distribution can be obtained by solving the master equation for the
system. The solution has a closed form and is given by:

$$
{\scriptstyle
P\left(m \mid k^{(p)}_{\text{on}}, k^{(p)}_{\text{off}}, r_m, \gamma_m\right) =
\frac{1}{\Gamma(m + 1)}
\frac{
    \Gamma
    \left(
        \frac{k^{(p)}_{\text{on}}}{\gamma_m} + m
    \right)
}{
    \Gamma
    \left(
        \frac{k^{(p)}_{\text{on}}}{\gamma_m}
    \right)
}
\frac{
    \Gamma
    \left(
        \frac{k^{(p)}_{\text{on}} + k^{(p)}_{\text{off}}}{\gamma_m}
    \right)
}{
    \Gamma
    \left(
        \frac{k^{(p)}_{\text{on}} + k^{(p)}_{\text{off}}}{\gamma_m} + m 
    \right)
}
\left( \frac{r_m}{\gamma_m} \right)^m \\
\times {}_1F_1 
\left(
    \frac{
            k^{(p)}_{\text{on}} 
        }{
            \gamma_m
        }
    + m,
    \frac{
            k^{(p)}_{\text{on}} + k^{(p)}_{\text{off}} 
        }{
            \gamma_m
        }
    + m,
    - \frac{r_m}{\gamma_m}
\right),
}
\tag{1}
$$


where $m$ is the steady-state mRNA copy number per cell, $k_{\text{on}}^{(p)}$
and $k_{\text{off}}^{(p)}$ are the rate constants for the promoter turning on
and off, respectively, $r_m$ is the mRNA production rate, and $\gamma_m$ is the
mRNA degradation rate.

The challenge to evaluate this distribution is that the confluent hypergeometric
function $_1F_1$ can be numerically unstable in certain parameter regimes. In a
separate notebook, we explored asymptotic approximations to the confluent
hypergeometric function, allowing us to tackle this problem. 



Here, we will focus
on setting up a neural network to perform inference on the parameters of the
model.

But before getting to that, let's show the steady-state mRNA distribution for
a parameter regime to make sure all functions previously-defined are working.

In [ ]:
# Define parameters
k_on = 4.0
k_off = 18.0
r_m = 100.0

# Define range of mRNA
m_range = np.arange(0, 75)

# Evaluate the log probability
logP = utils.two_state_log_probability(m_range, k_on, k_off, r_m)

# Initialize figure
fig, ax = plt.subplots(1, 1, figsize=(1.75, 1.5))

# Plot the probability
ax.step(m_range, np.exp(logP))

# Label axis
ax.set_xlabel("mRNA copy number")
ax.set_ylabel("Probability")

This looks excellent. Let's try again with a more challenging parameter regime.

In [ ]:
# Define parameters
k_on = 4.0
k_off = 2.0
r_m = 8000.0

# Define range of mRNA
m_range = np.arange(7000, 9000, 25)

# Evaluate the log probability
logP = utils.two_state_log_probability(m_range, k_on, k_off, r_m)

# Initialize figure
fig, ax = plt.subplots(1, 1, figsize=(1.75, 1.5))

# Plot the probability
ax.step(m_range, np.exp(logP) / np.sum(np.exp(logP)))

# Label axis
ax.set_xlabel("mRNA copy number")
ax.set_ylabel("Probability")

Again, everything looks in order. Let's now move on to setting the `BayesFlow`
neural network.

## Prior distribution function

Let's define the function that samples parameters from the prior distribution.

In [25]:
def prior_fn():
    # Prior on k_on rate
    k_on = np.abs(rng.normal(1E-4, 10))
    # Prior on k_off rate
    k_off = np.abs(rng.normal(1E-4, 10))
    # Prior on rate of transcription r
    r = 1E-3 + np.abs(10 * rng.standard_cauchy())

    return np.float32(np.array([k_on, k_off, r]))

We can now define the `BayesFlow` `simulation.Prior` object and sample from the
prior distribution.

In [ ]:
# Define parameter names
param_names = ['k_on', 'k_off', 'r']

# Define prior simulator
prior = bf.simulation.Prior(
    prior_fun=prior_fn,
    param_names=param_names,
)

# Draw samples from the prior
prior(10)

## Likelihood function

Next, we can define the likelihood function. This function takes as input the
log parameters and simulates a dataset by sampling from the likelihood function
conditioned on the parameters. The likelihood function is given by the steady
state mRNA distribution described above in Eq. (1).

### Naive likelihood function

For the first attempt, we will try to compute the likelihood function by 
evaluating the log probability of the mRNA copy number given the parameters for
a fixed range of mRNA copy numbers. After having these probabilities, we can
sample from the distribution to simulate UMI counts.

In [27]:
def likelihood_fn(params, n_obs=10_000, m_range=range(0, 2_000)):
    # Unpack parameters
    k_on, k_off, r = params
    # Compute the log probability over m_range
    logP = utils.two_state_log_probability(m_range, k_on, k_off, r)
    # Convert log probabilities to probabilities
    P = np.exp(logP)
    # Normalize the probabilities to use as weights. This is necessary because
    # of numerical precision issues.
    P /= P.sum()
    # Generate random samples using these weights
    u = np.random.choice(m_range, size=n_obs, p=P)
    # Add a 3rd dimension to the array to make output 3D tensor
    u = np.expand_dims(u, axis=1)
    # Return the samples as float32
    return np.float32(u)

Let's define the `BayesFlow` `simulation.Simulator` object and the generative
model.

In [ ]:
# Define Likelihood simulator function for BayesFlow
simulator = bf.simulation.Simulator(simulator_fun=likelihood_fn)

# Build generative model
model = bf.simulation.GenerativeModel(prior, simulator)

Next, let's time how long it takes to simulate 10 datasets.

In [ ]:
%time model_draws = model(10)

The time varies every time we run this simulation. The precision to compute the
confluent hypergeometric function is the main factor that determines the time it
takes to simulate the data.

Let's try simulating a large number of datasets.

In [ ]:
# Define the number of draws
n_draws = 100
# Draw samples from the generative model
%time model_draws = model(n_draws)

Now, let's plot the simulated data ECDFs.

In [ ]:
# Initialize figure
fig, ax = plt.subplots(1, 1, figsize=(1.75, 1.5))

# Loop through the draws and plot the data
for m in range(n_draws):
    sns.ecdfplot(model_draws["sim_data"][m, :, :].flatten(), ax=ax)

# Set x-axis to log scale
ax.set_xscale("log")

# Label axis
ax.set_xlabel("UMI counts")
ax.set_ylabel("ECDF")

This covers a wide range of mRNA copy numbers. 

## Summary network

Next, we define the summary network. For this, we will try the `DeepSet` network
architecture.

In [ ]:
# Define summary network as a Deepset
summary_net = bf.networks.DeepSet(summary_dim=8)

# Simulate a pass through the summary network
summary_pass = summary_net(model_draws["sim_data"])

summary_pass.shape

## Inference network

For the inference network, we will use the default `InvertibleNetwork`
architecture.

In [ ]:
# Define the conditional invertible network with affine coupling layers
inference_net = bf.inference_networks.InvertibleNetwork(
    num_params=prior(1)["prior_draws"].shape[-1],
)

inference_net(model_draws['prior_draws'], summary_pass)

## Training the model

Now we are ready to put everything together and train the model.

First, we define the `BayesFlow` `amortizer`.

In [54]:
# Assemble the amoratizer that combines the summary network and inference
# network
amortizer = bf.amortizers.AmortizedPosterior(
    inference_net, summary_net,
)

Next, we set the `trainer`.

In [ ]:
# Assemble the trainer with the amortizer and generative model
trainer = bf.trainers.Trainer(amortizer=amortizer, generative_model=model)

Now, we can train the model.

In [ ]:
# Define the number of epochs
n_epoch = 10
# Define the number of iterations per epoch
n_iter = 100
# Define the batch size
batch_size = 128
# Define number of validation simulations
n_val = 200

# Train the model
history = trainer.train_online(
    epochs=n_epoch,
    iterations_per_epoch=n_iter,
    batch_size=batch_size,
    validation_sims=n_val,
)

Let's save the model to disk.

In [60]:
amortizer.save_weights("./two_state_amortizer")

In [ ]:
f = bf.diagnostics.plot_losses(
    history["train_losses"], history["val_losses"], moving_average=True)

In [ ]:
test_sims = trainer.configurator(model(500))
z_samples, _ = amortizer(test_sims)
f = bf.diagnostics.plot_latent_space_2d(z_samples)

In [ ]:
# Obtain 100 posterior samples for each simulated data set in test_sims
posterior_samples = amortizer.sample(test_sims, n_samples=100)
f = bf.diagnostics.plot_sbc_histograms(
    posterior_samples, test_sims["parameters"], num_bins=10)

In [ ]:
f = bf.diagnostics.plot_sbc_ecdf(
    posterior_samples, test_sims["parameters"], difference=True)

In [ ]:
post_samples = amortizer.sample(test_sims, n_samples=1000)
f = bf.diagnostics.plot_recovery(post_samples, test_sims["parameters"])

In [ ]:
f = bf.diagnostics.plot_z_score_contraction(
    post_samples, test_sims["parameters"])